In [139]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.corpus import wordnet
from nltk import pos_tag


In [140]:
encodings=['utf-8','latin-1','ISO-8859-1','cp1252','ascii','utf-16','utf-32']

In [141]:
tweet_df=pd.read_csv("./src/data/annotated_data/Twitter_Annotated/Twitter_data_annotated_Lamia.csv")
df=tweet_df.copy()
df.head()

,id,text,created_at,user_id,Label
0,52629,- l'expérience des migrant·es avec @AntoineRob...,Sun Dec 18 10:00:43 +0000 2022,1.120000e+18,None
1,52448,"""Bonnes vacances avec les enfants ! Gros bisou...",Sun Dec 18 10:05:00 +0000 2022,4.146315e+08,None
2,53081,"""Chanter"" tu ne prie pas... un gros NTM SALE P...",Sun Dec 18 09:50:47 +0000 2022,1.550000e+18,Hate_Speech
3,53682,"""Elle dit qu'il y a des place vacantes,quand o...",Sun Dec 18 09:37:21 +0000 2022,8.470000e+17,None
4,52605,"""Tout niquer"". c'est classique chez un adolesc...",Sun Dec 18 10:01:20 +0000 2022,9.330000e+17,None


In [142]:
#EDA on the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2241 entries, 0 to 2240
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          2241 non-null   int64  
 1   text        2241 non-null   object 
 2   created_at  2241 non-null   object 
 3   user_id     2241 non-null   float64
 4   Label       2241 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 87.7+ KB


### EDA

In [ ]:
#preprocess the "text" column

def preprocess_usernames():
    #new column "usernames" to store the usernames
    df['usernames'] = df['text'].apply(lambda x: re.findall(r'@(\w+)', x))
    for i in range(len(df)):
        if len(df["usernames"][i]) > 0:
            df["usernames"][i] = df["usernames"][i][0]
        else:
            df["usernames"][i] = None

    #remove the usernames
    df['text'] = df['text'].apply(lambda x: re.sub(r'@(\w+)', '', x))


preprocess_usernames()


In [ ]:
def preprocess_urls():
    #new column "urls" to store the urls
    df['urls'] = df['text'].apply(lambda x: re.findall(r'http\S+', x))
    for i in range(len(df)):
        if len(df["urls"][i]) > 0:
            df["urls"][i] = df["urls"][i][0]
        else:
            df["urls"][i] = None

    #remove the urls
    df['text'] = df['text'].apply(lambda x: re.sub(r'http\S+', '', x))


preprocess_urls()


In [ ]:
def preprocess_hashtags():
    #new column "hashtags" to store the hashtags
    df['hashtags'] = df['text'].apply(lambda x: re.findall(r'#(\w+)', x))
    for i in range(len(df)):
        if len(df["hashtags"][i]) > 0:
            df["hashtags"][i] = df["hashtags"][i][0]
        else:
            df["hashtags"][i] = None

    #remove the hashtags
    df['text'] = df['text'].apply(lambda x: re.sub(r'#(\w+)', '', x))

preprocess_hashtags()    

In [146]:
#remove characters with different encodings 
df['text'] = df['text'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', x))

In [147]:
#new column "Day" to store the names of the day of the tweet
df["Day"]=df["created_at"].apply(lambda x: x.split(" ")[0])

#convet to datetime format
df["created_at"] = pd.to_datetime(df["created_at"])


In [148]:
#sepate the date and time
df["Date"]=df["created_at"].apply(lambda x: x.date())   
df["Time"]=df["created_at"].apply(lambda x: x.time())

#remove the "created_at" column
df.drop("created_at",axis=1,inplace=True)

In [149]:
# #count stopword in each tweet
# import pprint
# stop=[]
# for i in range(len(df)):
#     stop.append((f"tweet {i} has",len([word for word in df["text"][i].split() if word in stopwords.words('french')])))
# # pprint.pprint(stop)    

In [150]:
#remove special characters
# df['text'] = df['text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', ' ', x))


In [151]:
# df["text"].apply(lambda x: re.findall(f'@(\w+)', x)).value_counts()

In [152]:
df.head()

,id,text,user_id,Label,usernames,urls,hashtags,Day,Date,Time
0,52629,- l'exprience des migrantes avec \n,1.120000e+18,None,AntoineRoblain,https://t.co/x3KSTzppuj,None,Sun,2022-12-18,10:00:43
1,52448,"""Bonnes vacances avec les enfants ! Gros bisous""",4.146315e+08,None,None,https://t.co/5a9vADRtNQ,None,Sun,2022-12-18,10:05:00
2,53081,"""Chanter"" tu ne prie pas... un gros NTM SALE P...",1.550000e+18,Hate_Speech,None,https://t.co/1wv7ZotG8y,None,Sun,2022-12-18,09:50:47
3,53682,"""Elle dit qu'il y a des place vacantes,quand o...",8.470000e+17,None,None,https://t.co/oGeVRu7q7U,None,Sun,2022-12-18,09:37:21
4,52605,"""Tout niquer"". c'est classique chez un adolesc...",9.330000e+17,None,None,https://t.co/AjXyUUhL3p,None,Sun,2022-12-18,10:01:20
